REQUIRED DEPENDENCIES

In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

In [ ]:
%pip install  \
    "langchain>=0.0.350" \
    "transformers>=4.24,<5" \
    sqlalchemy -U \
    "faiss-cpu>=1.7,<2" \
    "pypdf>=3.8,<4" \
    pinecone-client==2.2.4 \
    apache-beam==2.52. \
    tiktoken==0.5.2 \
    "ipywidgets>=7,<8" \
    matplotlib==3.8.2 \
    anthropic==0.9.0

In [ ]:
%pip install pydantic==1.10.8

RESTART THE KERNEL

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

IMPORT BEDROCK CLIENT

Establishing a connection to the Berock service using AWS credentials and configuration settings provided through environment variables. This allows the Python script to interact with the Bedrock service using the boto3_bedrock object, which serves as a client for making API requests.

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

SET UP THE MODEL

In [ ]:
    from langchain.llms.bedrock import Bedrock

    inference_modifier = {'max_tokens_to_sample':4096, 
                        "temperature":0.5,
                        "top_k":250,
                        "top_p":1,
                        "stop_sequences": ["\n\nHuman"]
                        }

    textgen_llm = Bedrock(model_id = "anthropic.claude-v2",
                        client = boto3_bedrock, 
                        model_kwargs = inference_modifier 
                        )

# Prompting engineering
Prompting engineerins is the process of designing prompts in a way that optimizes the performance of large language models (LLMs) for specific tasks.
In this practice we are going to use the following prompting techniques :
- Zero-shot
- Few-shot
- COT

## Zero-shot prompt templates
Zero-shot prompt engineering involves crafting prompts for language models to enable them to perform tasks without explicit training examples or fine-tuning on that specific task, leveraging their initial training.

Large language models (LLMs) excel at various tasks out of the box:
- Text Generation: Generating text in a specific style or genre without fine-tuning on a large corpus of text in that style.
- Text Classification: Classifying text into predefined categories without explicit training examples for each category.
- Question Answering: Answering questions based on general knowledge without task-specific training data.
- Language Translation: Translating text between languages without training data for specific language pairs.
- Summarization: Generating summaries of text documents without task-specific training on summarization datasets.
- Sentiment Analysis: Analyzing the sentiment of text without labeled examples for each sentiment class.

Today's large LLMs, such as GPT-3.5, are finely tuned to follow instructions and are trained on vast amounts of data, making them capable of performing various tasks "zero-shot."

Here is one example of the Zero-shot techniques used for Sentiment Analysis :

In [15]:
prompt = """
Classify the text into neutral, negative or positive. 
Text: I think the vacation is okay.
Sentiment:
"""

response = textgen_llm(prompt)

print_ww(response)

Note that in the prompt above we didn't provide the model with any examples of text alongside their classifications, the LLM already understands "sentiment" -- that's the zero-shot capabilities at work.

### Exercises :
Try to solve the following problem using the Zero-shot techniques :

#### Exercise 1 - Text Summarization:
One of the standard tasks in natural language generation is text summarization. Text summarization can include many different flavors and domains. In fact, one of the most promising applications of language models is the ability to summarize articles and concepts into quick and easy-to-read summaries. Let's try a basic summarization task using prompts.

Let's say you are interested to learn about antibiotics.

Start by requesting an explanation of antibiotics from the model.
Then, ask the model to provide a concise summary of the definition it provided earlier.


In [ ]:
#Get the definition of antibiotics
prompt = """
...
"""

antibiotics_definition = textgen_llm(prompt)

#Summarize the definition of antibiotics
prompt_2 = antibiotics_definition + "..."
summary = textgen_llm(prompt_2)

print_ww(summary)

#### Exercise 2 - Code generation :
One application where LLMs are quite effective is code generation. Copilot is a great example of this. There are a vast number of code-generation tasks you can perform with clever prompts.

Imagine you're conducting an analysis across various departments within the university.
You need to extract the studentsID and the students names of the computer science department.

Provide info about possible database tables the model should use in the query.
Then ask the model to generate a valid MySQL query.

In [ ]:
#Get the definition of antibiotics
prompt = """
...
"""

SQL_query = textgen_llm(prompt)

print_ww(SQL_query)

## Few-shot prompt templates
While large-language models demonstrate remarkable zero-shot capabilities, they still fall short on more complex tasks when using the zero-shot setting. Few-shot prompting can be used as a technique to enable in-context learning where we provide demonstrations in the prompt to steer the model to better performance. The demonstrations serve as conditioning for subsequent examples where we would like the model to generate a response.

Let's demonstrate few-shot prompting via an example that was presented in [Brown et al. 2020](https://arxiv.org/abs/2005.14165)
. In the example, the task is to correctly use a new word in a sentence.

In [ ]:
prompt = """
A "whatpu" is a small, furry animal native to Tanzania. An example of a sentence that uses the word whatpu is:
We were traveling in Africa and we saw these very cute whatpus.
To do a "farduddle" means to jump up and down really fast. An example of a sentence that uses the word farduddle is:
"""

response = textgen_llm(prompt)

print_ww(response)


As you can see from the output the model understands the task and creates a valid example of use for an invented word.

Let's see another example :

In [ ]:
prompt ="""
This is awesome! // Negative
This is bad! // Positive
Wow that movie was rad! // Positive
What a horrible show! //
"""

response = textgen_llm(prompt)

print_ww(response)

#### Exercise - Reasoning:
Perhaps one of the most difficult tasks for an LLM today is one that requires some form of reasoning. Reasoning is one of most interesting areas due to the types of complex applications that can emerge from LLMs.

There have been some improvements in tasks involving mathematical capabilities. That said, it's important to note that current LLMs struggle to perform reasoning tasks so this requires even more advanced prompt engineering techniques. 

Given the prompt :
"The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"
The model should answer with True, if the statement is correct, False otherwise.

Check if the LLM can provide an accurate answer. If not, refine the prompt to assist the LLM in completing the task successfully.

In [ ]:
#Non-working solution with Few-shot technique
prompt = """

...

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A: 
"""

response = textgen_llm(prompt)

print_ww(response)

## COT : Chain of thoughts

Introduced in [Wei et al. (2022)](https://arxiv.org/abs/2201.11903), chain-of-thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps. You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding.

In [ ]:
prompt ="""
The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.
The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: Adding all the odd numbers (17, 19) gives 36. The answer is True.
The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: Adding all the odd numbers (11, 13) gives 24. The answer is True.
The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: Adding all the odd numbers (17, 9, 13) gives 39. The answer is False.
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:
"""


response = textgen_llm(prompt)

print_ww(response)

We can see a perfect result when we provided the reasoning step. In fact, we can solve this task by providing even fewer examples, i.e., just one example seems enough:

In [ ]:
prompt = """
The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:
"""


response = textgen_llm(prompt)

print_ww(response)

#### Exercise 1 - Math problem :
The next exercise present a prompt based on the Few-shot prompting technique. Since it involves reasoning it does not perform well.

Run the model with the given prompt to observe the answer and then try to improve it by using the COT technique.

In [ ]:
#Example with Few-shot prompting
prompt = """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls.
How many tennis balls does he have now?

A: The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
"""

response = textgen_llm(prompt)

print_ww(response)

#Correct solution
prompt = """
...
"""

response = textgen_llm(prompt)

print_ww(response)

#### Exercise 2 - Commonsense:

Another area where COT can excel is in addressing common sense questions.
Change the prompt with few-shot COT technique and try to make the model answer correctly.


In [ ]:
prompt = """

...

Q: Google Maps and other highway and street GPS services have replaced what? Answer Choices: (a) united
states (b) mexico (c) countryside (d) atlas
A:
"""

response = textgen_llm(prompt)

print_ww(response)

#### Exercise 3 - StrategyQA:

Another area where COT can excel is in addressing strategcal reasoning question.
Change the prompt with few-shot COT technique and try to make the model answer correctly.


In [ ]:
prompt = """

...

Q: Yes or no: Hydrogen’s atomic number squared exceeds number of Spice Girls?
A:
"""

response = textgen_llm(prompt)

print_ww(response)

#### Exercise 4 - Sports understanding:

Another area where COT can excebe exploited is in make the model understand terminologies of a specific field
that may commonly have other meanings.
Change the prompt with few-shot COT technique and try to make the model answer correctly.


In [ ]:
prompt = """

...

Q: Is the following sentence plausible? “Draymond Green threw a touchdown.”
A:
"""

response = textgen_llm(prompt)

print_ww(response)

#### Exercise 5 - Last letter concatenation:

Another area where COT can excel is in task concerning carachters manipulation.
Change the prompt with few-shot COT technique and try to make the model answer correctly.


In [ ]:
prompt = """
Q: Take the last letters of the words
in “Lady Gaga” and concatenate
them.
A:
"""

response = textgen_llm(prompt)

print_ww(response)

## Zero-shot COT : 

One recent idea that came out more recently is the idea of zero-shot CoT [(Kojima et al. 2022)]() that essentially involves adding "Let's think step by step" to the original prompt. Let's try a simple problem and see how the model performs:

In [ ]:
#prompt with Zero-shot COT
prompt = """
I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman.
I then went and bought 5 more apples and ate 1. How many apples did I remain with?
Let's think step by step.
"""

response = textgen_llm(prompt)

print_ww(response)

--------------------------------------------------------------------
                                    SEE BELOW FOR EXERCISES SOLUTIONS
--------------------------------------------------------------------

## SOLUTIONS

### Zero-shot

In [ ]:
# Exercise 1

#Get the definition of antibiotics
prompt = """
"Explain antibiotics\n"
"A:"
"""

antibiotics_definition = textgen_llm(prompt)

#Summarize the definition of antibiotics
prompt_2 = antibiotics_definition + "\nExplain the above text in one sentence :"
summary = textgen_llm(prompt_2)

print_ww(summary)

In [ ]:
#### Exercise 2

#Get the definition of antibiotics
prompt = """
    Table departments, columns = [DepartmentId, DepartmentName]
    Table students, columns = [DepartmentId, StudentId, StudentName]
    Create a MySQL query for all students in the Computer Science Department
    """

SQL_query = textgen_llm(prompt)

print_ww(SQL_query)

### Few-shot


In [ ]:
#### Exercise 1
#Solution with Few-shot technique
prompt = """
The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The answer is False.
The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: The answer is True.
The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: The answer is True.
The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: The answer is False.
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A: 
"""

response = textgen_llm(prompt)

print_ww(response)

### COT - Chain of thaugths

In [ ]:
#### Exercise 1

#Example with Few-shot prompting
prompt = """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls.
How many tennis balls does he have now?
A: The answer is 11.
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
"""

response = textgen_llm(prompt)

print_ww(response)

In [ ]:
#Exercise 2
prompt = """
Q: What home entertainment equipment requires cable?
Answer Choices: (a) radio shack (b) substation (c) television (d) cabinet
A: The answer must require cable. Of the above choices, only television requires cable. So the answer is (c).

Q: The fox walked from the city into the forest, what was it looking for? Answer Choices: (a) pretty flowers (b)
hen house (c) natural habitat (d) storybook
A: The answer must be something in the forest. Of the above choices, only natural habitat is in the forest. So the
answer is (b).

Q: Google Maps and other highway and street GPS services have replaced what? Answer Choices: (a) united
states (b) mexico (c) countryside (d) atlas
A:
"""

response = textgen_llm(prompt)

print_ww(response)

In [ ]:
#Exercise 3
prompt = """
Q: Yes or no: Is it common to see frost during some college commencements?
A: College commencement ceremonies can happen in December, May, and June. December is in the winter, so
there can be frost. Thus, there could be frost at some commencements. So the answer is yes.

Q: Yes or no: Could a llama birth twice during War in Vietnam (1945-46)?
A: The War in Vietnam was 6 months. The gestation period for a llama is 11 months, which is more than 6
months. Thus, a llama could not give birth twice during the War in Vietnam. So the answer is no.

Q: Yes or no: Hydrogen’s atomic number squared exceeds number of Spice Girls?
A:
"""

response = textgen_llm(prompt)

print_ww(response)

In [ ]:
#Exercise 4
prompt = """
Q: Is the following sentence plausible? “Kyle Palmieri was called for slashing.”
A: Kyle Palmieri is a hockey player. Being called for slashing is part of hockey. So the answer is yes.

Q: Is the following sentence plausible? “Carson Wentz set the pick and roll.”
A: Carson Wentz is an American football player. Pick and roll is part of basketball, not football. So the answer
is no.

Q: Is the following sentence plausible? “Draymond Green threw a touchdown.”
A:
"""

response = textgen_llm(prompt)

print_ww(response)

In [ ]:
#Exercise 5
prompt = """
Q: Take the last letters of the words in "Elon Musk" and concatenate them.
A: The last letter of "Elon" is "n". The last letter of "Musk" is "k". Concatenating them is "nk". The answer is nk.

Q: Take the last letters of the words in "Larry Page" and concatenate them.
A: The last letter of "Larry" is "y". The last letter of "Page" is "e". Concatenating them is "ye". The answer is ye

Q: Take the last letters of the words
in “Lady Gaga” and concatenate
them.
A:
"""

response = textgen_llm(prompt)

print_ww(response)